<a href="https://colab.research.google.com/github/somilasthana/deeplearningtutorial/blob/master/Saving_Restoring_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q h5py pyyaml 

In [0]:
from __future__ import absolute_import, division, print_function
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images, train_label), (test_images, test_label) = tf.keras.datasets.mnist.load_data()
train_images.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

In [0]:
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28)/255.0

In [7]:
train_images.shape

(1000, 784)

In [0]:
train_labels = train_label[:1000]
test_labels = test_label[:1000]

In [0]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
      keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10, activation=tf.nn.softmax)  
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
  
  return model

In [15]:
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [0]:
!mkdir training

In [0]:
checkpoint_path = "./training/cp.cpkt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 period=2, # save every 2 epoch
                                                 verbose=1
                                                )

model = create_model()
model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

In [18]:
!ls {checkpoint_dir}

checkpoint  cp.cpkt.data-00000-of-00001  cp.cpkt.index


In [23]:
# Load the Model
model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 98us/sample - loss: 2.3934 - acc: 0.1030
Untrained model, accuracy: 10.30%


In [24]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Trained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 47us/sample - loss: 0.4244 - acc: 0.8570
Trained model, accuracy: 85.70%


In [0]:
#Manually Saving

# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))